# **Atenção**

## Para executar esse notebook utilize o T4 GPU

Além disso, a saída de algumas células estão comentadas, visto que sua exibição implica na utilização de memória RAM do Colab, gerando erro de processamento. Os códigos estão funcionando, porém precisam de maior capacidade computacional para rodarem sem problemas. Na próxima sprint espera-se conseguir aprimorar as questões de eficiência.

# Integrantes

- Dayllan Alho
- Giovanna Furlan
- Luiz Ferreira
- Maria Luísa Maia
- Thainá Lima
- Vinicius Fernandes

# Introdução

## **Problema a ser resolvido**

O projeto tem como objetivo desenvolver uma aplicação para a Aegea, uma empresa de saneamento, que melhore a detecção de fraudes no consumo de água. Essas fraudes, como a manipulação de hidrômetros e ligações clandestinas, não só prejudicam o faturamento e a qualidade do serviço, mas também podem causar danos à infraestrutura e representar riscos à saúde pública.

## **Solução Proposta**

Para identificar as fraudes, foi proposto o desenvolvimento de uma aplicação baseada em Machine Learning que melhore a capacidade da Aegea de detectar comportamentos fraudulentos no consumo de água. Essa solução considera a influência de variáveis externas, como fatores econômicos, climáticos e geográficos, para aumentar a assertividade na identificação e melhorar os processos de negócio da empresa.

# Setup

A configuração de setup é o processo de preparar e organizar o ambiente para uso. Envolvendo a instalação de bibliotecas e configuração de outros ajustes necessários. O objetivo é criar um ambiente funcional para executar tarefas específicas.

## Instalação das bibliotecas

 Nesta seção, são instaladas e importadas as bibliotecas necessárias para a manipulação, análise e visualização dos dados.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from google.colab import drive
from scipy import stats
import os
import gdown
import requests
from datetime import datetime
import plotly.express as px
import json
from pandas import DataFrame
from typing import List
from sklearn.preprocessing import OneHotEncoder


## **Importação da base de dados**

Para realizar a análise, padronização e manipulação dos dados é necessário selecionar a base de dados desejada. Neste documento a importação da mesma será feita através do gdown e o arquivo está em formato CSV.

In [ ]:
arquivo_base = "dataset_{}.csv"

ids = {
    "CONSUMO_2024": "1-2YBshCNQI4pOMV83BGJnTrCFpDTcv_h",
    "CONSUMO_2023": "1WmLtlgEgMvAqJ9kjxP_xT0oecsLaM_vr",
    "CONSUMO_2022": "1-8l0qgFq40NhqZbMM_gBYJO32vSbr0eG",
    "CONSUMO_2021": "12-fFTkYIDOIorDnj3zAWwynNPC6B8mb2",
    "CONSUMO_2020": "1-1pOoa0eJlNJ94BMi7p4PTx5KUS96mhX",
    "CONSUMO_2019": "1MRCFBXGPkdkEe546LbrOw3ypJRKi4ofG",
    "FRAUDES": "1Yils9N2z5V_8SJLrmxtiIbauWFJ3WmWm",
}

for key, file_id in ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_base.format(key)

    gdown.download(url, arquivo_destino, quiet=False)

In [ ]:
arquivos_csv = [
    "./dataset_CONSUMO_2024.csv",
    "./dataset_CONSUMO_2023.csv",
    "./dataset_CONSUMO_2022.csv",
    "./dataset_CONSUMO_2021.csv",
    "./dataset_CONSUMO_2020.csv",
    "./dataset_CONSUMO_2019.csv"
]

CONSUMO_GERAL = pd.concat([pd.read_csv(arquivo, delimiter=",") for arquivo in arquivos_csv], axis=0)

# Lendo o arquivo de fraudes
FRAUDES = pd.read_csv("./dataset_FRAUDES.csv", delimiter=",")

## **Visualização dos tipos das colunas**

As colunas apresentadas nas tabelas disponibilizadas possuem diferentes tipos de dados, sendo divididos em:

1. **float**: Responsável por armazenar números reais com casas decimais, utilizados para representar medidas ou valores contínuos.
2. **int**: Responsável por armazenar números inteiros, normalmente utilizados para contagens ou identificadores.
3. **object**: Utilizado para armazenar dados genéricos, como strings, representando características categóricas ou textuais.

Como as colunas seguem o mesmo padrão, apresenta-se o exemplo de dtypes com o Consumo de 2024.

In [ ]:
CONSUMO_GERAL.dtypes

In [ ]:
FRAUDES.dtypes

# Processamento e Transformação dos dados

## Exclusão de colunas

Devido às limitações de capacidade de RAM no Colab, e em conversa com o orientador, foi sugerido subir os dados já com as colunas removidas para tornar o carregamento mais leve. Essa decisão foi tomada considerando que a exclusão dessas colunas já está contemplada na análise dos dados.

In [ ]:
CONSUMO_GERAL = CONSUMO_GERAL.drop(columns=["SUB_CATEGORIA"])

## Transformação dos dados da coluna MATRICULA do Consumo Geral

Isso foi feito no intuito de deixar a coluna Matricula tabela de consumo_geral e para que o dado seja mais leve para a memória ram

In [ ]:
CONSUMO_GERAL["MATRICULA"] = CONSUMO_GERAL["MATRICULA"].astype(int)

# mudando para int para ter uma eficiencia melhor de memoria
CONSUMO_GERAL["CONS_MEDIDO"] = CONSUMO_GERAL["CONS_MEDIDO"].astype(int)

## Verificação de valores nulos

Processo importante para identificar dados ausentes no dataset. Esses valores podem afetar a precisão dos modelos e a qualidade das análises.

In [ ]:
def verificar_quant_de_nulos(df, nome):
  print(f"\nValores nulos em {nome}:")
  print(df.isnull().sum())

# Verificar valores nulos - CONSUMO_GERAL
verificar_quant_de_nulos(CONSUMO_GERAL, "CONSUMO_GERAL")

# Verificar valores nulos - FRAUDES
verificar_quant_de_nulos(FRAUDES, "FRAUDES")

## Tratamento de células vazias

Algumas colunas do dataset continham valores nulos, o que poderia comprometer o pré-processamento e a manipulação dos dados. Para solucionar esse problema, foi implementada uma função que preenche esses valores nulos de maneira apropriada, dependendo do tipo de dado em cada coluna.

In [ ]:
class FillNullProcessor:
    @staticmethod
    def fill_missing_values_consumo_geral(df):
        # Coordenadas padrão do centro de Mato Grosso
        latitude_centro_MT = -12.6819
        longitude_centro_MT = -56.9211

        # Substituindo valores nulos em COD_LATITUDE e COD_LONGITUDE com as coordenadas padrão
        df["COD_LATITUDE"] = df["COD_LATITUDE"].fillna(latitude_centro_MT)
        df["COD_LONGITUDE"] = df["COD_LONGITUDE"].fillna(longitude_centro_MT)

        # Convertendo para tipo numérico (float)
        df["COD_LATITUDE"] = pd.to_numeric(df["COD_LATITUDE"], errors='coerce')
        df["COD_LONGITUDE"] = pd.to_numeric(df["COD_LONGITUDE"], errors='coerce')

        return df

    @staticmethod
    def fill_missing_values_fraudes(df):
        # Substituindo valores nulos em colunas numéricas e categóricas com 0
        df["COD_GRUPO"] = df["COD_GRUPO"].fillna(0)
        df["SETOR"] = df["SETOR"].fillna(0)
        df["DESCRSETORSOLICITANTE"] = df["DESCRSETORSOLICITANTE"].fillna(0)
        df["CD_SUB_REGIAO"] = df["CD_SUB_REGIAO"].fillna(0.0)
        df["CD_REGIAO"] = df["CD_REGIAO"].fillna(0.0)
        df["PARECER_EXECUCAO"] = df["PARECER_EXECUCAO"].fillna(0)

        # Convertendo a coluna de datas para o formato datetime antes de preencher os nulos
        df["DT_FECHAMENTO"] = pd.to_datetime(df["DT_FECHAMENTO"], errors='coerce')

        # Preenchendo valores nulos na coluna DT_FECHAMENTO com 0
        df["DT_FECHAMENTO"] = df["DT_FECHAMENTO"].fillna(0)

        # Convertendo colunas numéricas e categóricas para tipos numéricos apropriados
        df["COD_GRUPO"] = pd.to_numeric(df["COD_GRUPO"], errors='coerce')
        df["SETOR"] = pd.to_numeric(df["SETOR"], errors='coerce')
        df["DESCRSETORSOLICITANTE"] = pd.to_numeric(df["DESCRSETORSOLICITANTE"], errors='coerce')
        df["CD_SUB_REGIAO"] = pd.to_numeric(df["CD_SUB_REGIAO"], errors='coerce')
        df["CD_REGIAO"] = pd.to_numeric(df["CD_REGIAO"], errors='coerce')
        df["PARECER_EXECUCAO"] = pd.to_numeric(df["PARECER_EXECUCAO"], errors='coerce')

        return df

# Aplicando a função ao DataFrame FRAUDES
FRAUDES = FillNullProcessor.fill_missing_values_fraudes(FRAUDES)
CONSUMO_GERAL = FillNullProcessor.fill_missing_values_consumo_geral(CONSUMO_GERAL)

In [ ]:
# Verificando se ainda existem valores nulos nos DataFrames
print("Verificação de valores nulos em CONSUMO_GERAL:")
print(CONSUMO_GERAL.isnull().sum())

print("\nVerificação de valores nulos em FRAUDES:")
print(FRAUDES.isnull().sum())

## Engenharia de recursos

A engenharia de recursos envolve a criação e transformação de  features para melhorar a performance e a interpretabilidade do modelo. O objetivo é extrair informações mais relevantes e úteis dos dados brutos, adaptando-os para o modelo, como os realizados abaixo.

In [ ]:
# Multiplicar e converter a coluna CONS_MEDIDO em uma nova coluna

# Verificar esse código a baixo:
CONSUMO_GERAL['CONS_MEDIDO_EST_ANUAL'] = (CONSUMO_GERAL['CONS_MEDIDO'] * 12).astype('float32')

# Converter a coluna REFERENCIA para datetime
CONSUMO_GERAL['REFERENCIA'] = pd.to_datetime(CONSUMO_GERAL['REFERENCIA'], format='%Y-%m-%d')

### Detecção de Anomalias no Padrão de Consumo Geográfico

Pensamos em calcular estatísticas agregadas para o consumo em cada região (com base em latitude e longitude), e então identificar anomalias no consumo. Por exemplo, se o consumo de água em uma área específica está muito acima ou abaixo da média da área.

**Observação:** Como neste momento de pré-processamento o código abaixo não está sendo utilizado, ele ficou comentado para evitar utilizar espaço na RAM e implicar em erro de processamento.

In [ ]:
# # Criando uma chave geográfica
# CONSUMO_GERAL['geo_key'] = CONSUMO_GERAL['COD_LATITUDE'].round(3).astype(str) + "_" + CONSUMO_GERAL['COD_LONGITUDE'].round(3).astype(str)

# # Calculando o consumo médio por região (agrupado pela chave geográfica)
# consumo_medio_geo = CONSUMO_GERAL.groupby('geo_key')['CONS_MEDIDO'].mean()

# # Comparando o consumo de cada ponto com a média da região
# CONSUMO_GERAL['diferenca_media_geo'] = CONSUMO_GERAL['CONS_MEDIDO'] - CONSUMO_GERAL['geo_key'].map(consumo_medio_geo)

# # Identificando possíveis fraudes se a diferença for muito grande (fora de 3 desvios padrões, por exemplo)
# desvio_padrao_geo = CONSUMO_GERAL.groupby('geo_key')['CONS_MEDIDO'].std()

# # Nesta parte há a possibilidade de adicionar uma variável a mais
# #CONSUMO_GERAL['is_anomaly'] = (CONSUMO_GERAL['diferenca_media_geo'].abs() > 3 * CONSUMO_GERAL['geo_key'].map(desvio_padrao_geo))

## Divisão das Categorias

Foi realizada a divisão das categorias para a análise dos dados, estruturando-os nos grupos principais:

1. **Categorias de Consumo**:
   - **Residencial**: Relacionada ao consumo em residências.
   - **Comercial**: Refere-se ao consumo em estabelecimentos comerciais.
   - **Pública**: Abrange o consumo em prédios e serviços públicos.
   - **Industrial**: Inclui o consumo em instalações industriais.

2. **Categorias de Ligação:**

- **Hidômetro:** Refere-se ao consumo medido por hidrômetro, que é o dispositivo utilizado para medir a quantidade de água consumida.
- **Consumo Fixo:** Relacionado a uma estimativa ou tarifa fixa de consumo que não varia com o uso.

Essa divisão visa proporcionar uma análise mais segmentada, permitindo análise dos padrões de consumo e possíveis irregularidades.

In [ ]:
class DataSeparatorHandler:
    @staticmethod
    def get_consumo_residencial(df):
        # Divisão dos dados em categorias usando groupby
        consumos = df.groupby('CATEGORIA')

        # Acesso cada categoria conforme necessário
        consumo_residencial = consumos.get_group('RESIDENCIAL')
        consumo_comercial = consumos.get_group('COMERCIAL')
        consumo_publica = consumos.get_group('PUBLICA')
        consumo_industrial = consumos.get_group('INDUSTRIAL')

        return consumo_residencial, consumo_comercial, consumo_publica, consumo_industrial

# Aplicando a função e verificando as saídas
consumo_residencial, consumo_comercial, consumo_publica, consumo_industrial = DataSeparatorHandler.get_consumo_residencial(CONSUMO_GERAL)

# Exibindo as primeiras linhas de cada DataFrame
print("Consumo Residencial:")
print(consumo_residencial.head())

In [ ]:
print("Consumo Comercial:")
print(consumo_comercial.head())

In [ ]:
print("Consumo Pública:")
print(consumo_publica.head())

In [ ]:
print("Consumo Industrial:")
print(consumo_industrial.head())

In [ ]:
# Divisão dos dados em categorias usando groupby
tipos_ligacao = CONSUMO_GERAL.groupby('TIPO_LIGACAO')

# Acesso cada categoria conforme necessário
consumo_hidrometrado = tipos_ligacao.get_group('Hidrometrado')
consumo_fixo = tipos_ligacao.get_group('Consumo Fixo')

In [ ]:
# Salvando os DataFrames em arquivos CSV
CONSUMO_GERAL.to_csv("/content/drive/Shareddrives/LeakSeeker/DADOS_MODELO/consumo_geral.csv", index=False)
FRAUDES.to_csv("/content/drive/Shareddrives/LeakSeeker/DADOS_MODELO/fraudes.csv", index=False)
